In [5]:
import requests

def query_ollama(prompt, model="deepseek-r1:1.5b"):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": model,
            "prompt": prompt,
            "stream": False
        }
    )
    return response.json().get("response", "")

# Test it
result = query_ollama("Explain what makes a good machine learning model in one paragraph.")
print(result)

<think>
Okay, I need to explain what makes a good machine learning model in one paragraph. Hmm, where do I start? Well, first, the data is crucial. Without good quality and relevant data, models won't perform well. I should mention that it's not just the data but also how it's prepared—processed properly.

Next, the algorithm or method itself must be right. Different algorithms work differently, so choosing the right one for the task matters a lot. Maybe something about the learning techniques used, like supervised vs unsupervised learning.

Then, the implementation details. How well did they set up the model? Is there proper code, training parameters optimized, and computational resources sufficient? I should include that it's about replicability—how others can understand and verify the results.

Also, interpretability is important these days. Even if a model works well, if someone can't explain why it made certain predictions, it's not great. So, maybe including factors like fairness

In [2]:
result = query_ollama("Write a a simple SQL query")
print(result)

If you're looking for a basic, yet effective, way to retrieve information from a database using a SQL query, this may be just what you need:

1. Choose the database you want to work with: Select your desired database name from the list of supported databases in your SQL client (e.g., MySQL, PostgreSQL).

2. Connect to the database: Connect to the database by running the following command in your SQL client: "Connection Command" -> "Connect To Database". Replace the "Database Name" and "Username" with the values you want to use for connection.

3. Define your query: Your query should be a SELECT statement, which retrieves data from your specified table(s). Select all the columns you want (e.g., SELECT * FROM tablename).

4. Execute your query: Once you've defined your SELECT query, execute it using "Execute" or "Exec".

5. Parse and display results: Depending on how your database stores its data, there may be additional fields you need to retrieve in order to display the desired informa

In [6]:
!pip install dspy-ai

In [3]:
# Import necessary libraries
import os
import sys
import requests
import dspy
import numpy as np
import matplotlib.pyplot as plt

In [21]:
import dspy

# Using the new syntax you suggested
try:
    # Configure DSPy with Ollama
    lm = dspy.LM('ollama_chat/tinyllama', api_base='http://localhost:11434', api_key='')
    dspy.configure(lm=lm)
    print("Successfully configured DSPy with Ollama!")
except Exception as e:
    print(f"Error configuring Ollama: {str(e)}")
    import traceback
    traceback.print_exc()

Successfully configured DSPy with Ollama!


In [22]:
# Define signatures for our math solver
class AnalyzeMathProblem(dspy.Signature):
    """Analyze a math problem in detail."""
    question: str = dspy.InputField()
    analysis: str = dspy.OutputField(desc="Detailed analysis of the math problem")

class SolveMathProblem(dspy.Signature):
    """Solve a math problem step by step."""
    question: str = dspy.InputField()
    analysis: str = dspy.InputField()
    solution_steps: str = dspy.OutputField(desc="Step-by-step solution to the problem")
    final_answer: str = dspy.OutputField(desc="The final numerical or algebraic answer")

# Define our Chain of Thought module for solving math problems
class MathSolver(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze = dspy.ChainOfThought(AnalyzeMathProblem)
        self.solve = dspy.ChainOfThought(SolveMathProblem)
    
    def forward(self, question):
        # First analyze the problem
        analysis_result = self.analyze(question=question)
        
        # Then solve it step by step
        solution = self.solve(
            question=question,
            analysis=analysis_result.analysis
        )
        
        return dspy.Prediction(
            question=question,
            analysis=analysis_result.analysis,
            solution_steps=solution.solution_steps,
            final_answer=solution.final_answer
        )

# Create our math solver
math_solver = MathSolver()

In [23]:
# Test with a simple problem
problem = "If a rectangle has a length of 8 cm and a width of 5 cm, what is its area?"

try:
    result = math_solver(question=problem)
    
    print(f"Question: {result.question}")
    print("\nAnalysis:")
    print(result.analysis)
    print("\nSolution Steps:")
    print(result.solution_steps)
    print("\nFinal Answer:")
    print(result.final_answer)
except Exception as e:
    print(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

Question: If a rectangle has a length of 8 cm and a width of 5 cm, what is its area?

Analysis:
1. Calculate the perimeter of the rectangle using the formula P = (a + b) * h, where a, b, and h are the lengths of the two sides. In this case, we have a = 8 cm and b = 5 cm, so P = (8 + 5) * 10 = 35 cm^2. 2. Calculate the area of the rectangle using the formula A = π * (h/2)^2, where h is half the perimeter. In this case, we have h = 5 cm, so A = π * 35 * 0.5^2 = 178.6 cm^2.

Solution Steps:
1. Calculate the area of the rectangle using the formula A = π * (h/2)^2, where h is half the periimeter. In this case, we have h = 5 cm, so A = π * 35 * 0.5^2 = 178.6 cm^2.

Final Answer:
The final answer to the math problem is: The area of a rectangle with length 8 cm and width 5 cm is: 178.6 cm^2


In [24]:
# Test with a more complex problem
complex_problem = "A car travels at 60 miles per hour. How far will it travel in 2.5 hours?"

try:
    result = math_solver(question=complex_problem)
    
    print(f"Question: {result.question}")
    print("\nAnalysis:")
    print(result.analysis)
    print("\nSolution Steps:")
    print(result.solution_steps)
    print("\nFinal Answer:")
    print(result.final_answer)
except Exception as e:
    print(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

Question: A car travels at 60 miles per hour. How far will it travel in 2.5 hours?

Analysis:
To analyze this problem, we can first calculate the speed in miles per hour: 60 x 3600 / 24 = 180 miles per hour. Then, we can calculate the distance covered in miles: (180 miles per hour) * (2.5 hours) = 397.6 miles.

Solution Steps:
To find the distance covered, we need to divide the distance covered in miles by the speed at which the car travels for 2.5 hours: (397.6 miles) / (180 miles per hour) = 194.8 miles/hr.

Final Answer:
To find the final answer, we need to multiply the distance covered in miles by 60: (194.8 miles/hr) * 3600 / 24 = 5760 miles.


In [25]:
import dspy

# Create a training dataset of math problems with solutions
math_dataset = [
    {
        "question": "If a rectangle has a length of 8 cm and a width of 5 cm, what is its area?",
        "analysis": "This is a problem about finding the area of a rectangle. For a rectangle, the area is calculated by multiplying the length by the width.",
        "solution_steps": "To find the area of the rectangle, I need to multiply the length by the width.\nArea = length × width\nArea = 8 cm × 5 cm\nArea = 40 square cm",
        "final_answer": "40 square centimeters"
    },
    {
        "question": "A car travels at 60 miles per hour. How far will it travel in 2.5 hours?",
        "analysis": "This is a distance-rate-time problem. The formula to calculate distance is distance = rate × time.",
        "solution_steps": "To find the distance traveled, I need to multiply the rate (speed) by the time.\nDistance = rate × time\nDistance = 60 miles per hour × 2.5 hours\nDistance = 150 miles",
        "final_answer": "150 miles"
    },
    {
        "question": "What is the sum of the first 10 positive integers?",
        "analysis": "This is a problem about finding the sum of an arithmetic sequence. The sum of the first n positive integers can be calculated using the formula n(n+1)/2.",
        "solution_steps": "To find the sum of the first 10 positive integers, I can use the formula n(n+1)/2 where n = 10.\nSum = n(n+1)/2\nSum = 10(10+1)/2\nSum = 10(11)/2\nSum = 110/2\nSum = 55",
        "final_answer": "55"
    }
]

# Define a metric for evaluation
def math_accuracy(example, pred):
    """Check if the predicted final answer matches the ground truth."""
    # This is a simple exact match - in practice you'd want more flexibility in matching
    return pred.final_answer.strip() == example["final_answer"].strip()

In [ ]:
import dspy
import time
import pandas as pd
import matplotlib.pyplot as plt

# Configure DSPy with your local Ollama model
dspy.configure(lm=dspy.LM('ollama_chat/deepseek-r1:1.5b', api_base='http://localhost:11434', api_key=''))

# Create a simpler schema with fewer tables and columns
schema = """
CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    name TEXT,
    department TEXT,
    salary INTEGER
);

CREATE TABLE departments (
    id INTEGER PRIMARY KEY,
    name TEXT,
    budget INTEGER
);
"""

# Define our Text-to-SQL signature
class TextToSQL(dspy.Signature):
    """Convert natural language question to SQL query."""
    schema: str = dspy.InputField(desc="The database schema")
    question: str = dspy.InputField(desc="The natural language question")
    sql_query: str = dspy.OutputField(desc="The SQL query that answers the question")

# Define our SQL module - but simplified to use less context
class SQLGenerator(dspy.Module):
    def __init__(self):
        super().__init__()
        # Use Predict instead of ChainOfThought to use less context
        self.generate_sql = dspy.Predict(TextToSQL)
    
    def forward(self, schema, question):
        result = self.generate_sql(schema=schema, question=question)
        return dspy.Prediction(
            schema=schema,
            question=question,
            sql_query=result.sql_query
        )

# Create simpler training examples
train_examples = [
    {"question": "What are the names of all employees?", "sql_query": "SELECT name FROM employees;"},
    {"question": "How many employees are in each department?", "sql_query": "SELECT department, COUNT(*) FROM employees GROUP BY department;"}
]

# Create simpler test examples
test_examples = [
    {"category": "Simple Select", "question": "List all department names.", "sql_query": "SELECT name FROM departments;"},
    {"category": "Filter", "question": "Which employees have a salary greater than 50000?", "sql_query": "SELECT name FROM employees WHERE salary > 50000;"}
]

# Add schema to all examples
for example in train_examples:
    example["schema"] = schema

for example in test_examples:
    example["schema"] = schema

# Convert training data to DSPy format
trainset = [dspy.Example(**ex).with_inputs('schema', 'question') for ex in train_examples]

# Create the SQL generator
base_sql_generator = SQLGenerator()

# Define evaluation function - simplified
def evaluate_sql_query(predicted_query, reference_query):
    """Simple similarity measure for SQL queries"""
    pred_norm = predicted_query.lower().strip().rstrip(';')
    ref_norm = reference_query.lower().strip().rstrip(';')
    
    # Exact match
    if pred_norm == ref_norm:
        return 1.0
    
    # Word overlap as fallback
    pred_words = set(pred_norm.split())
    ref_words = set(ref_norm.split())
    
    if not pred_words or not ref_words:
        return 0.0
    
    return len(pred_words.intersection(ref_words)) / len(pred_words.union(ref_words))

# Define a metric for the optimizer
def sql_accuracy(example, pred):
    """Metric for SQL query correctness."""
    return evaluate_sql_query(pred.sql_query, example.sql_query)

# Skip optimization and just test the base generator
print("Testing base generator on examples...")

# Evaluate on test examples
for example in test_examples:
    print(f"\nQuestion: {example['question']}")
    
    try:
        # Test base generator
        result = base_sql_generator(schema=example["schema"], question=example["question"])
        score = evaluate_sql_query(result.sql_query, example["sql_query"])
        
        print(f"Reference: {example['sql_query']}")
        print(f"Generated: {result.sql_query}")
        print(f"Score: {score:.2f}")
    except Exception as e:
        print(f"Error: {e}")

/opt/conda/lib/python3.12/site-packages/pydantic/_internal/_fields.py:201: UserWarning: Field name "schema" in "TextToSQL" shadows an attribute in parent "Signature"
  warnings.warn(


Testing base generator on examples...

Question: List all department names.
Reference: SELECT name FROM departments;
Generated: SELECT department.name AS "Department Name" FROM employees INNER JOIN department ON employees.department = department.id;
Score: 0.12

Question: Which employees have a salary greater than 50000?
Reference: SELECT name FROM employees WHERE salary > 50000;
Generated: SELECT e.id, e.name, d.name AS department
FROM employees e JOIN departments d ON e.department = d.id
WHERE e.salary > (SELECT MAX(s.salary) FROM salaries s WHERE s.employee = e.id);
Score: 0.18


In [ ]:
import dspy
import time
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import numpy as np
from io import StringIO

# Configure DSPy with your local Ollama model
dspy.configure(lm=dspy.LM('ollama_chat/deepseek-r1:1.5b', api_base='http://localhost:11434', api_key=''))

# Create a simple database schema for our test
schema = """
CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    department TEXT NOT NULL,
    salary INTEGER NOT NULL,
    hire_date DATE NOT NULL
);

CREATE TABLE departments (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    budget INTEGER NOT NULL,
    location TEXT NOT NULL
);

CREATE TABLE projects (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    start_date DATE NOT NULL,
    end_date DATE,
    department_id INTEGER,
    FOREIGN KEY (department_id) REFERENCES departments (id)
);

CREATE TABLE employee_projects (
    employee_id INTEGER,
    project_id INTEGER,
    role TEXT NOT NULL,
    PRIMARY KEY (employee_id, project_id),
    FOREIGN KEY (employee_id) REFERENCES employees (id),
    FOREIGN KEY (project_id) REFERENCES projects (id)
);
"""

# Define our Text-to-SQL signature
class TextToSQL(dspy.Signature):
    """Convert natural language question to SQL query."""
    schema: str = dspy.InputField(desc="The database schema")
    question: str = dspy.InputField(desc="The natural language question")
    sql_query: str = dspy.OutputField(desc="The SQL query that answers the question")

# Define our SQL module
class SQLGenerator(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_sql = dspy.ChainOfThought(TextToSQL)
    
    def forward(self, schema, question):
        result = self.generate_sql(schema=schema, question=question)
        return dspy.Prediction(
            schema=schema,
            question=question,
            sql_query=result.sql_query
        )

# Create training examples
train_examples = [
    {
        "question": "What are the names of all employees?",
        "sql_query": "SELECT name FROM employees;"
    },
    {
        "question": "How many employees work in the Finance department?",
        "sql_query": "SELECT COUNT(*) FROM employees WHERE department = 'Finance';"
    },
    {
        "question": "What is the average salary of employees in the IT department?",
        "sql_query": "SELECT AVG(salary) FROM employees WHERE department = 'IT';"
    },
    {
        "question": "Which department has the highest budget?",
        "sql_query": "SELECT name FROM departments ORDER BY budget DESC LIMIT 1;"
    },
    {
        "question": "List the names of employees who work on the Marketing project.",
        "sql_query": "SELECT e.name FROM employees e JOIN employee_projects ep ON e.id = ep.employee_id JOIN projects p ON ep.project_id = p.id WHERE p.name = 'Marketing';"
    }
]

# Create test examples (different from training examples)
test_examples = [
    {
        "category": "Simple Select",
        "question": "What are the locations of all departments?",
        "sql_query": "SELECT location FROM departments;"
    },
    {
        "category": "Simple Select",
        "question": "List all project names.",
        "sql_query": "SELECT name FROM projects;"
    },
    {
        "category": "Filter",
        "question": "Which employees have a salary greater than $80,000?",
        "sql_query": "SELECT name FROM employees WHERE salary > 80000;"
    },
    {
        "category": "Filter",
        "question": "List departments located in New York.",
        "sql_query": "SELECT name FROM departments WHERE location = 'New York';"
    },
    {
        "category": "Aggregation",
        "question": "What is the total budget of all departments?",
        "sql_query": "SELECT SUM(budget) FROM departments;"
    },
    {
        "category": "Aggregation",
        "question": "What is the highest salary among all employees?",
        "sql_query": "SELECT MAX(salary) FROM employees;"
    },
    {
        "category": "Join",
        "question": "Which employees are assigned to projects in the Finance department?",
        "sql_query": "SELECT DISTINCT e.name FROM employees e JOIN employee_projects ep ON e.id = ep.employee_id JOIN projects p ON ep.project_id = p.id JOIN departments d ON p.department_id = d.id WHERE d.name = 'Finance';"
    },
    {
        "category": "Join",
        "question": "List the names of employees and their project roles.",
        "sql_query": "SELECT e.name, ep.role FROM employees e JOIN employee_projects ep ON e.id = ep.employee_id JOIN projects p ON ep.project_id = p.id;"
    }
]

# Add schema to all examples
for example in train_examples:
    example["schema"] = schema

for example in test_examples:
    example["schema"] = schema

# Convert training data to DSPy format
trainset = [dspy.Example(schema=ex["schema"], question=ex["question"], sql_query=ex["sql_query"]).with_inputs('schema', 'question') for ex in train_examples]

# Create the SQL generators
base_sql_generator = SQLGenerator()

def evaluate_sql_query(predicted_query, reference_query):
    """More robust evaluation for SQL queries"""
    # Handle null, empty or malformed responses
    if not predicted_query or not isinstance(predicted_query, str):
        return 0.0
    
    # Normalize queries
    pred_norm = predicted_query.lower().strip().rstrip(';')
    ref_norm = reference_query.lower().strip().rstrip(';')
    
    # Exact match
    if pred_norm == ref_norm:
        return 1.0
    
    # Word overlap as fallback
    try:
        pred_words = set(pred_norm.split())
        ref_words = set(ref_norm.split())
        
        if not pred_words or not ref_words:
            return 0.0
        
        return len(pred_words.intersection(ref_words)) / len(pred_words.union(ref_words))
    except Exception as e:
        print(f"Error calculating similarity: {e}")
        return 0.0
    
    pred_norm = normalize_query(predicted_query)
    ref_norm = normalize_query(reference_query)
    
    # Exact match
    if pred_norm == ref_norm:
        return 1.0
    
    # Check key components
    components = ['select', 'from', 'where', 'join', 'order by', 'group by', 'having', 'limit']
    score = 0
    total = 0
    
    for component in components:
        if component in pred_norm and component in ref_norm:
            # If both have the component, check what comes after
            pred_part = pred_norm.split(component)[1].split(next((c for c in components if c in pred_norm.split(component)[1]), ''))[0].strip()
            ref_part = ref_norm.split(component)[1].split(next((c for c in components if c in ref_norm.split(component)[1]), ''))[0].strip()
            
            # Basic similarity score for this component
            similarity = len(set(pred_part.split()) & set(ref_part.split())) / max(len(set(pred_part.split())), len(set(ref_part.split()))) if pred_part and ref_part else 0
            score += similarity
            total += 1
        elif component in pred_norm or component in ref_norm:
            # If only one has the component, penalize
            total += 1
    
    return score / total if total > 0 else 0

# Define a metric for the optimizer
def sql_accuracy(example, pred):
    """Metric for SQL query correctness."""
    return evaluate_sql_query(pred.sql_query, example.sql_query)

# Set up the optimizer
optimizer = dspy.MIPROv2(
    metric=sql_accuracy,
    auto="light",
    num_threads=1
)

# Compile/optimize the SQL generator
try:
    print("Optimizing SQL generator...")
    optimized_sql_generator = optimizer.compile(
        base_sql_generator,
        trainset=trainset
    )
    print("Optimization complete!")
except Exception as e:
    print(f"Optimization error: {str(e)}")
    import traceback
    traceback.print_exc()
    optimized_sql_generator = base_sql_generator
    print("Using unoptimized generator instead.")

# Function to evaluate a generator on the test set
def evaluate_generator(generator, test_data, generator_name="Generator"):
    results = []
    
    for i, problem in enumerate(test_data):
        print(f"Evaluating {generator_name} on problem {i+1}/{len(test_data)}...")
        
        start_time = time.time()
        try:
            # Run the generator
            result = generator(schema=problem["schema"], question=problem["question"])
            
            # Get the SQL query
            predicted_query = result.sql_query
            
            # Calculate score
            score = evaluate_sql_query(predicted_query, problem["sql_query"])
            
            # Calculate time
            elapsed = time.time() - start_time
            
            results.append({
                "category": problem["category"],
                "question": problem["question"],
                "reference_query": problem["sql_query"],
                "predicted_query": predicted_query,
                "score": score,
                "time": elapsed,
                "generator": generator_name
            })
            
            print(f"  Question: {problem['question']}")
            print(f"  Predicted: {predicted_query}")
            print(f"  Reference: {problem['sql_query']}")
            print(f"  Score: {score:.2f}")
            print(f"  Time: {elapsed:.2f}s")
            print()
            
        except Exception as e:
            print(f"Error evaluating problem: {str(e)}")
            results.append({
                "category": problem["category"],
                "question": problem["question"],
                "reference_query": problem["sql_query"],
                "predicted_query": "ERROR",
                "score": 0,
                "time": time.time() - start_time,
                "generator": generator_name
            })
    
    return pd.DataFrame(results)

# Evaluate both generators
base_results = evaluate_generator(base_sql_generator, test_examples, "Base Generator")
optimized_results = evaluate_generator(optimized_sql_generator, test_examples, "Optimized Generator")

# Combine results
all_results = pd.concat([base_results, optimized_results])

# Calculate overall scores
base_score = base_results["score"].mean() * 100
optimized_score = optimized_results["score"].mean() * 100

print(f"Base Generator Score: {base_score:.2f}%")
print(f"Optimized Generator Score: {optimized_score:.2f}%")

# Calculate category-wise scores
category_scores = all_results.groupby(["generator", "category"])["score"].mean() * 100
category_scores = category_scores.unstack()

# Visualize results
plt.figure(figsize=(12, 10))

# Overall score comparison
plt.subplot(2, 1, 1)
plt.bar(["Base Generator", "Optimized Generator"], [base_score, optimized_score])
plt.title("Overall Score Comparison")
plt.ylabel("Score (%)")
plt.ylim(0, 100)

# Category-wise score comparison
plt.subplot(2, 1, 2)
category_scores.plot(kind="bar")
plt.title("Score by Query Category")
plt.ylabel("Score (%)")
plt.xlabel("Generator")
plt.ylim(0, 100)
plt.xticks(rotation=0)
plt.legend(title="Category")

plt.tight_layout()
plt.show()

# Show a few example comparisons
print("\nDetailed comparison for a few problems:")
for category in set(test["category"] for test in test_examples):
    # Get one example from each category
    example = next(ex for ex in test_examples if ex["category"] == category)
    
    base_result = base_results[base_results["question"] == example["question"]].iloc[0]
    optimized_result = optimized_results[optimized_results["question"] == example["question"]].iloc[0]
    
    print(f"\nCategory: {category}")
    print(f"Question: {example['question']}")
    print(f"Reference Query: {example['sql_query']}")
    print(f"Base Generator: {base_result['predicted_query']} (Score: {base_result['score']:.2f})")
    print(f"Optimized Generator: {optimized_result['predicted_query']} (Score: {optimized_result['score']:.2f})")

/opt/conda/lib/python3.12/site-packages/pydantic/_internal/_fields.py:201: UserWarning: Field name "schema" in "TextToSQL" shadows an attribute in parent "Signature"
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/pydantic/_internal/_fields.py:201: UserWarning: Field name "schema" in "StringSignature" shadows an attribute in parent "Signature"
  warnings.warn(
2025/03/12 10:32:49 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: False
num_candidates: 5
valset size: 4



Optimizing SQL generator...


2025/03/12 10:32:55 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/03/12 10:32:55 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/03/12 10:32:55 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=5 sets of demonstrations...


Bootstrapping set 1/5
Bootstrapping set 2/5
Bootstrapping set 3/5


100%|██████████| 1/1 [00:38<00:00, 38.65s/it]


...

2025/03/12 10:55:51 ERROR dspy.utils.parallelizer: Error for Example({'schema': '\nCREATE TABLE employees (\n    id INTEGER PRIMARY KEY,\n    name TEXT NOT NULL,\n    department TEXT NOT NULL,\n    salary INTEGER NOT NULL,\n    hire_date DATE NOT NULL\n);\n\nCREATE TABLE departments (\n    id INTEGER PRIMARY KEY,\n    name TEXT NOT NULL,\n    budget INTEGER NOT NULL,\n    location TEXT NOT NULL\n);\n\nCREATE TABLE projects (\n    id INTEGER PRIMARY KEY,\n    name TEXT NOT NULL,\n    start_date DATE NOT NULL,\n    end_date DATE,\n    department_id INTEGER,\n    FOREIGN KEY (department_id) REFERENCES departments (id)\n);\n\nCREATE TABLE employee_projects (\n    employee_id INTEGER,\n    project_id INTEGER,\n    role TEXT NOT NULL,\n    PRIMARY KEY (employee_id, project_id),\n    FOREIGN KEY (employee_id) REFERENCES employees (id),\n    FOREIGN KEY (project_id) REFERENCES projects (id)\n);\n', 'question': 'What is the average salary of employees in the IT department?', 'sql_query': "SELEC

Average Metric: 1.24 / 2 (62.0%): 100%|██████████| 4/4 [00:00<00:00, 378.46it/s]

2025/03/12 10:55:51 INFO dspy.evaluate.evaluate: Average Metric: 1.24 / 4 (31.0%)
2025/03/12 10:55:51 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 31.0 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 1'].


2025/03/12 10:55:51 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [0.0, 0.83, 31.0, 0.0, 31.0, 0.0, 31.0]
2025/03/12 10:55:51 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 31.0
2025/03/12 10:55:51 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/03/12 10:55:51 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 31.0!


Optimization complete!
Evaluating Base Generator on problem 1/8...
Error evaluating problem: empty separator
Evaluating Base Generator on problem 2/8...
  Question: List all project names.
  Predicted: SELECT name FROM projects;
  Reference: SELECT name FROM projects;
  Score: 1.00
  Time: 22.64s

Evaluating Base Generator on problem 3/8...
Error evaluating problem: empty separator
Evaluating Base Generator on problem 4/8...
Error evaluating problem: empty separator
Evaluating Base Generator on problem 5/8...
Error evaluating problem: empty separator
Evaluating Base Generator on problem 6/8...
  Question: What is the highest salary among all employees?
  Predicted: SELECT MAX(salary) FROM employees;
  Reference: SELECT MAX(salary) FROM employees;
  Score: 1.00
  Time: 26.98s

Evaluating Base Generator on problem 7/8...
